In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold,train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE

# Used Imputed csv file that has been made by us in imputation_code.csv

In [2]:
app_train = pd.read_csv('application_train_imputed.csv')
y = app_train['TARGET']

In [3]:
X=app_train.drop(['TARGET'],axis=1)
X.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
# Remove above line of Unnamed: 0 after making changes to application_train_imputed.csv

In [4]:
X.head()

,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,...,DAYS_ID_PUBLISH,OWN_CAR_AGE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_LAST_PHONE_CHANGE
0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-2120.0,12.061091,1.0,2.0,2.0,10.0,0.083037,0.262949,0.139376,-1134.0
1,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-291.0,12.061091,2.0,1.0,1.0,11.0,0.311267,0.622246,0.510853,-828.0
2,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-2531.0,26.000000,1.0,2.0,2.0,9.0,0.502130,0.555912,0.729567,-815.0
3,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2437.0,12.061091,2.0,2.0,2.0,17.0,0.502130,0.650442,0.510853,-617.0
4,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-3458.0,12.061091,1.0,2.0,2.0,11.0,0.502130,0.322738,0.510853,-1106.0


In [5]:
y.head()

0    1.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: TARGET, dtype: float64

In [6]:

X_train_later, X_test, y_train_later, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)
print("Postive examples in train set: {}".format(np.sum(y_train_later==0)))
print("Negative examples in train set: {}".format(np.sum(y_train_later==1)))

print("Postive examples in test set: {}".format(np.sum(y_test==0)))
print("Negative examples in test set: {}".format(np.sum(y_test==1)))

Postive examples in train set: 226148
Negative examples in train set: 19860
Postive examples in test set: 56538
Negative examples in test set: 4965


In [7]:
X_train, X_val, y_train, y_val = train_test_split(X_train_later, y_train_later, test_size=0.2, stratify=y_train_later, random_state=1)


In [8]:
ratio = (y == 0).sum()/ (y == 1).sum()
ratio

11.387150050352467

In [9]:
# hist = xgb.cv(params, dtrain, num_rounds, nfold=10, stratified=True, early_stopping_rounds=10, verbose_eval=True)

In [10]:
clf1 = XGBClassifier(n_estimators=190, objective='binary:logistic', 
                    gamma=0.1,subsample=0.5, scale_pos_weight=ratio )
clf1.fit(X_train_later, y_train_later)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0.1, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=190,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=11.387150050352467,
       seed=None, silent=True, subsample=0.5)

In [11]:
y_pred=clf1.predict(X_test)
newdf=pd.DataFrame(data=y_pred)
newdf.iloc[:,0].value_counts()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.0    41118
1.0    20385
Name: 0, dtype: int64

In [12]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

        0.0       0.96      0.70      0.81     56538
        1.0       0.17      0.69      0.27      4965

avg / total       0.90      0.70      0.77     61503



In [13]:
confusion_matrix(y_test, y_pred)

array([[39556, 16982],
       [ 1562,  3403]], dtype=int64)

# Part 2: Used Smote + Part 1

In [9]:
sm = SMOTE(random_state=12, ratio = 1.0)
X_train_res, y_train_res = sm.fit_sample(X_train_later, y_train_later)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


In [10]:
print("Postive examples in train set: {}".format(np.sum(y_train_res==0)))
print("Negative examples in train set: {}".format(np.sum(y_train_res==1)))


Postive examples in train set: 226148
Negative examples in train set: 226148


### Below code left deliberately with Error so that I can explain Why I renamed columns in Test in 3 rd cell from below to some weird names. Juts that SMOTE creates this types of labels and We have to train on that SMOTE data, I needed to give this types of names just to match feature names and Successfully it worked

In [11]:
clf2 = XGBClassifier(n_estimators=190, objective='binary:logistic', 
                    gamma=0.1,subsample=0.5, scale_pos_weight=ratio )
clf2.fit(X_train_res,y_train_res)
y_pred=clf2.predict(X_test)

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', '

In [15]:
X_test.shape

(61503, 245)

In [16]:
X_test.columns=['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', 'f134', 'f135', 'f136', 'f137', 'f138', 'f139', 'f140', 'f141', 'f142', 'f143', 'f144', 'f145', 'f146', 'f147', 'f148', 'f149', 'f150', 'f151', 'f152', 'f153', 'f154', 'f155', 'f156', 'f157', 'f158', 'f159', 'f160', 'f161', 'f162', 'f163', 'f164', 'f165', 'f166', 'f167', 'f168', 'f169', 'f170', 'f171', 'f172', 'f173', 'f174', 'f175', 'f176', 'f177', 'f178', 'f179', 'f180', 'f181', 'f182', 'f183', 'f184', 'f185', 'f186', 'f187', 'f188', 'f189', 'f190', 'f191', 'f192', 'f193', 'f194', 'f195', 'f196', 'f197', 'f198', 'f199', 'f200', 'f201', 'f202', 'f203', 'f204', 'f205', 'f206', 'f207', 'f208', 'f209', 'f210', 'f211', 'f212', 'f213', 'f214', 'f215', 'f216', 'f217', 'f218', 'f219', 'f220', 'f221', 'f222', 'f223', 'f224', 'f225', 'f226', 'f227', 'f228', 'f229', 'f230', 'f231', 'f232', 'f233', 'f234', 'f235', 'f236', 'f237', 'f238', 'f239', 'f240', 'f241', 'f242', 'f243', 'f244']

In [17]:
y_pred=clf2.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [18]:
newdf=pd.DataFrame(data=y_pred)
newdf.iloc[:,0].value_counts()

0.0    38343
1.0    23160
Name: 0, dtype: int64

In [19]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

        0.0       0.96      0.65      0.78     56538
        1.0       0.15      0.72      0.25      4965

avg / total       0.90      0.66      0.74     61503



In [20]:
confusion_matrix(y_test, y_pred)

array([[36928, 19610],
       [ 1415,  3550]], dtype=int64)

In [22]:
import seaborn as sns
top40 = pd.DataFrame({'features': X_train.columns, 'importance': clf2.feature_importances_}).sort_values('importance', ascending=False).head(40)
# sns.barplot(x=top50['importance'], y=top50['features'])

In [23]:
top40

,features,importance
243,EXT_SOURCE_3,0.091765
242,EXT_SOURCE_2,0.069020
98,NAME_INCOME_TYPE_Working,0.056471
241,EXT_SOURCE_1,0.046275
228,AMT_CREDIT,0.035294
233,DAYS_EMPLOYED,0.028235
4,FLAG_PHONE,0.027451
77,CODE_GENDER_F,0.021961
103,NAME_EDUCATION_TYPE_Secondary / secondary special,0.021176
100,NAME_EDUCATION_TYPE_Higher education,0.021176


In [10]:
clf4 = XGBClassifier(n_estimators=180, objective='binary:logistic',
                    gamma=0.1, subsample=0.5, scale_pos_weight=ratio )
clf4.fit(X_train, y_train)
# clf3.fit(X_train,y_train)
y_pred=clf4.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [12]:
newdf=pd.DataFrame(data=y_pred)
newdf.iloc[:,0].value_counts()

0.0    40885
1.0    20618
Name: 0, dtype: int64

In [13]:

print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

        0.0       0.96      0.70      0.81     56538
        1.0       0.17      0.69      0.27      4965

avg / total       0.90      0.70      0.76     61503



In [15]:
confusion_matrix(y_test, y_pred)

array([[39346, 17192],
       [ 1539,  3426]], dtype=int64)